In [36]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [5]:
all_models = np.load('all_models_reg_n_grade.npy')
best_model = all_models[2]
print best_model

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)


In [69]:
#Define a function to one-hot encode data using the type of columns
#INPUT: Dataframe
#       Columns with categorical values
#OUTPUT: Encoded dataset
#Main functionality: Convert all categorical columns to independent columns and
#                    Standardize all parameters
def encode(data, encode_cols):
    final_cols = ['y', 'Verified', 'n', 'car', 'credit_card', 'debt_consolidation', 'educational', 'home_improvement', 
                  'Source Verified', 'house', 'medical', 'moving', 'other', 'renewable_energy', 'small_business', 'vacation', 
                  'wedding', 'major_purchase', 'RENT', 'Not Verified', 'OTHER', '36 months', '60 months', '1 year', '10+ years', 
                  '2 years', '3 years', 'OWN', '5 years', '6 years', '4 years', '8 years', '9 years', '< 1 year', 'n/a', 
                  'MORTGAGE', '7 years', 'pub_rec_bankruptcies', 'pub_rec', 'delinq_2yrs', 'inq_last_6mths', 'curr_mean_fico', 
                  'sub_grade', 'open_acc', 'earliest_cr_line', 'prev_mean_fico', 'total_acc', 'int_rate', 'funded_amnt', 
                  'revol_util', 'dti', 'annual_inc', 'revol_bal']
    encoded_x = pd.DataFrame({})
    encoded_final_x = pd.DataFrame(columns=final_cols)
    
    for col in data.columns:
        
        if (col in encode_cols[:]):
            encoded = pd.get_dummies(data[col])
            encoded_x = pd.concat([encoded_x, encoded], axis = 1)
        else:
            data[col] = data[col].astype(float)
            #data[col] = (data[col] - data[col].min())/(data[col].max() - data[col].min())
            encoded_x = pd.concat([encoded_x, data[[col]]], axis = 1)
            
    for col in encoded_x.columns:
        encoded_final_x[col] = encoded_x[col]
        
    encoded_final_x = encoded_final_x.fillna(0.0)
    
    return encoded_final_x

In [78]:
#input_list = [5000.00, '36 months', 10.65, 'B2', '10+ years','RENT', 24000, 'Verified',
#              'n','credit_card', 27.65, 0, 'Jan-85',1 , 3, 0, 
#              13648, 83.7,9, 0, 737, 742]

input_list = [200000, '60 months', 50.89, 'F1', '6 years','RENT', 70000, 'Source Verified',
              'n','debt_consolidation', 16.63, 0, 'Nov-92', 10, 10, 0, 
              10470, 79.9, 50, 0, 450, 542]  


all_cols = ['funded_amnt', 'term', 'int_rate', 'sub_grade', 'emp_length','home_ownership', 'annual_inc', 'verification_status',
           'pymnt_plan','purpose', 'dti', 'delinq_2yrs', 'earliest_cr_line','inq_last_6mths', 'open_acc', 'pub_rec', 
           'revol_bal', 'revol_util','total_acc', 'pub_rec_bankruptcies', 'curr_mean_fico','prev_mean_fico']
encode_cols = ['emp_length', 'home_ownership', 'verification_status', 'term', 'pymnt_plan', 'purpose']


input_df = pd.DataFrame(columns = all_cols)
input_df.loc[0] = input_list

input_df['grade'] = input_df['sub_grade'][0][0]
input_df['grade'] = input_df['grade'].map({'A': 0, 'B': 5, 'C': 10, 'D': 15, 'E': 20, 'F': 25, 'G': 30})
input_df['sub_grade'] = input_df['sub_grade'].str.extract('(\d+)').astype(int) + input_df['grade']
input_df.drop('grade', 1, inplace=True)

input_df.earliest_cr_line = pd.to_datetime(input_df.earliest_cr_line)
date_diff = (pd.datetime.today() - input_df.earliest_cr_line)
input_df.earliest_cr_line = date_diff.dt.days/365
input_df.earliest_cr_line = input_df.earliest_cr_line.astype(int)

final_df = encode(input_df, encode_cols)
print final_df

print best_model.predict(final_df)
# Web needs to take % away from int_rate, revol_util
# leave month as it is given in file MMM-YY (provide proper input format)
#


     y  Verified    n  car  credit_card  debt_consolidation  educational  \
0  0.0       0.0  1.0  0.0          0.0                 1.0          0.0   

   home_improvement  Source Verified  house    ...      open_acc  \
0               0.0              1.0    0.0    ...          10.0   

   earliest_cr_line  prev_mean_fico  total_acc  int_rate  funded_amnt  \
0              24.0           542.0       50.0     50.89     200000.0   

   revol_util    dti  annual_inc  revol_bal  
0        79.9  16.63     70000.0    10470.0  

[1 rows x 53 columns]
[0]
